# Topic Modelling

In [1]:
import pandas as pd
import gensim

C:\Users\keiko\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
beige=pd.read_csv('noSS_table.csv')
beige.head()

,Unnamed: 0,ID,Date,District,Sector Heading,nostop_stem
0,0,1,5-20-1970,Atlanta,Summary of Economic Activity,"['mood', 'director', 'vary', 'pessimism', 'opt..."
1,1,2,5-20-1970,Boston,Summary of Economic Activity,"['discussion', 'monday', '18', 'friday', 'dist..."
2,2,3,5-20-1970,Chicago,Summary of Economic Activity,"['publish', 'statement', 'private', 'conversat..."
3,3,4,5-20-1970,Cleveland,Summary of Economic Activity,"['economic', 'activity', 'district', 'disrupt'..."
4,4,5,5-20-1970,Dallas,Summary of Economic Activity,"['continue', 'inflation', 'major', 'concern', ..."


In [5]:
type(list(beige['nostop_stem'])[0])

str

In [3]:
corpus = [eval(beige['nostop_stem'][i]) for i in range(len(beige))]
type(corpus[:3][0])

list

In [4]:
#download en on spaCy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [5]:
def tablefier(parsed_text):
    toks_tab = pd.DataFrame()
    toks_tab['Word'] = [word.text for word in parsed_text]
    toks_tab['POS'] = [word.pos_ for word in parsed_text]
    return toks_tab

In [ ]:
parsed = nlp(beige['nostop_stem'][0])

In [21]:
beige['nostop_stem'][0]

str

In [53]:
gram = tablefier(parsed)
gram= gram[gram['POS'] != 'PUNCT']
gram=gram[gram['POS']!='ADP']

In [54]:
gram

,Word,POS
2,mood,NOUN
6,director,NOUN
10,vary,VERB
14,pessimism,NOUN
18,optimism,NOUN


In [6]:
one = pd.read_csv('noSS_onelist.csv')


In [7]:
ones = one.groupby('string').count()

In [8]:
ones.head()

,Unnamed: 0
string,
,420
,3
,1
-PRON-,5
0,30


In [9]:
print('total number of different words that appear once: ', len(ones[ones['Unnamed: 0'] == 1]))

total number of different words that appear once:  24967


In [10]:
print('total number of unique words: ',len(ones))

total number of unique words:  40405


In [11]:
print('total number of unique words appearing more than once: ', len(ones)-24967)

total number of unique words appearing more than once:  15438


In [12]:
oncewords = list(ones[ones['Unnamed: 0'] ==1].index)

In [13]:
beige.head()

,Unnamed: 0,ID,Date,District,Sector Heading,nostop_stem
0,0,1,5-20-1970,Atlanta,Summary of Economic Activity,"['mood', 'director', 'vary', 'pessimism', 'opt..."
1,1,2,5-20-1970,Boston,Summary of Economic Activity,"['discussion', 'monday', '18', 'friday', 'dist..."
2,2,3,5-20-1970,Chicago,Summary of Economic Activity,"['publish', 'statement', 'private', 'conversat..."
3,3,4,5-20-1970,Cleveland,Summary of Economic Activity,"['economic', 'activity', 'district', 'disrupt'..."
4,4,5,5-20-1970,Dallas,Summary of Economic Activity,"['continue', 'inflation', 'major', 'concern', ..."


In [ ]:
a = [1,2, 3, 4, 5]
type(str(a))
str(a)

In [ ]:
new_corp= []
for t in beige['nostop_stem']:
    new = [word for word in t if word not in oncewords]
    parsed = nlp(str(new))
    gram = tablefier(parsed)
    gram = gram[(gram['POS'] == 'NOUN')&(gram['POS']=='VERB')&(gram['POS']=='ADJ')]
    new_corp.append(list(gram['Word']))

In [ ]:
beige['improved'] = new_corp

In [ ]:
beige.to_csv(path_or_buf = 'improved.csv', sep = ',', encoding='utf-8')

In [ ]:
beige['improved'].head()

In [5]:
from gensim import corpora

In [3]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(corpus)

NameError: name 'corpora' is not defined

In [8]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus]

In [9]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

In [10]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=1)

In [2]:
print(ldamodel.print_topics(num_topics=30, num_words=10))

NameError: name 'ldamodel' is not defined